In [1]:
import os
import pandas as pd
import json
import random
import shutil
import numpy as np
import pathlib
import pickle

In [2]:
source_pose_dir = './raw_data/UBnormal/poses'
source_annotations_dir = './raw_data/UBnormal/annotations'

In [34]:
dst_pose_dir = './TrajREC/data/UBnormal/testing/trajectories'
dst_annotations_dir = './TrajREC/data/UBnormal/testing/frame_level_masks'

In [8]:
def get_jsons(directory: str):
    """Function that exctracts the mp4 files from the given directory
    and returns the path to the video and it's annotations
    """
    filelist = [];
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('tracked_person.json'):
                json_file = os.path.join(root,file)
                filelist.append(json_file)
    return filelist

def get_mp4s(directory: str):
    """Function that exctracts the mp4 files from the given directory
    and returns the path to the video and it's annotations
    """
    filelist = [];
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.mp4'):
                json_file = os.path.join(root,file)
                filelist.append(json_file)
    return filelist

def remove_files(directory: str):
    try:
        shutil.rmtree(directory)
    except FileNotFoundError:
        pass

def convert_lines_annotation_to_mask(annotations):
    filters = []
    for line in annotations:
        _, start_frame, end_frame = line.split(',')
        # Use float to avoid issues with scientific notation
        filters.append(
                (
                    int(float(start_frame)),
                    int(float(end_frame))
                )
        )
    
    mask = [1] * 451
    for i in range(451):
        for start, end in filters:
            if start <= i <= end:
                mask[i] = 0
    return mask


In [6]:
json_files = get_jsons(source_pose_dir)
# mp4_files = get_mp4s(source_video_dir)
# mp4_dict = {f.split('/')[-1].split('.')[0]: f for f in mp4_files}

In [68]:
remove_files(dst_pose_dir)

# Avoid issue due to invalid track.txt format
test_scenes = []
while len(test_scenes) < 25:
    scene = random.choices(json_files, k=1)[0]

    # Validate that the scene contains a trajectory
    with open(scene) as f:
        tracking = json.load(f)
        if not tracking:
            continue
    pose_file_name = scene.split('/')[-1]
    video_name = '_'.join(pose_file_name.split('-')[0].split('_')[0:-3])
    # Check if the annotation file exists
    annotation_file = f'./raw_data/UBnormal/annotations/{video_name}_tracks.txt'
    try:
        np.load(annotation_file, allow_pickle=True)
    except pickle.UnpicklingError:
        # The files with a different format will be handled down the line
        pass
    except FileNotFoundError:
        continue
    if scene not in test_scenes:
        test_scenes.append(scene)

In [64]:
remove_files(dst_pose_dir)
remove_files(dst_annotations_dir)

In [69]:
# File structure:
# testing/trajectories/video_id/*.csv
# video used in the repo is {flag}{scene_number}_{scenarion_number}
# 1 csv per detected person
remove_files(dst_pose_dir)
remove_files(dst_annotations_dir)
for pose_file_name in test_scenes:
    video_name = '_'.join(pose_file_name.split('/')[-1].split('-')[0].split('_')[0:-3])
    
    with open(pose_file_name) as f:
        tracking = json.load(f)
    data_dict = {}
    for traj_id, traj in tracking.items():
        csv_data = []
        for frame, data in traj.items():
            csv_record = [frame]
            key_points = data['keypoints']
            for i in range(0, len(key_points), 3):
                x = float(key_points[i])
                y = float(key_points[i+1])
                # Skip processing the confidence
                csv_record.append(x)
                csv_record.append(y)
            csv_data.append(csv_record)
        data_dict[traj_id] = csv_data
    for traj_id, csv_data in data_dict.items():
        # Skip records with a short trajectory
        if len(csv_data) < 12:
            continue
        df = pd.DataFrame(csv_data)
        track_dir = f'{dst_pose_dir}/{video_name.replace("_", "")}_{traj_id}'
        pathlib.Path(track_dir).mkdir(parents=True, exist_ok=True)
        track_target_file = f'{track_dir}/{video_name.replace("_", "")}_{traj_id}.csv'
        # print(f'Writing trajectory to {track_target_file}')
        df.to_csv(track_target_file, index=False)

        # Copy annotation:
        annotation_file = f'./raw_data/UBnormal/annotations/{video_name}_tracks.txt'
        try:
            mask = np.load(annotation_file, allow_pickle=True)
        except pickle.UnpicklingError:
            with open(annotation_file) as f:
                annotations = [line.strip() for line in f.readlines()]
            mask = convert_lines_annotation_to_mask(annotations)
        annotation_dir = f'{dst_annotations_dir}/{video_name.replace("_", "")}_{traj_id}'
        pathlib.Path(annotation_dir).mkdir(parents=True, exist_ok=True)
        annotations_file = f'{annotation_dir}/{video_name.replace("_", "")}_{traj_id}.npy'
        np.save(file=annotations_file, arr=mask, allow_pickle=True)